Desafio:
1. Carregar um arquivo csv com feedback de clientes (reviews.csv)
2. Usar LLM pra classificar o sentimento de cada feedback (Positivo, Negativo, Neutro)
3. Adicionar classificação ao DF

In [2]:
#!pip install google-genai
import pandas as pd

In [3]:
df = pd.read_csv('data/reviews.csv')
df.head()

,reviewerID,asin,reviewerName,reviewText,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A2FII3I2MBMUIA,B007WTAJTO,1K3,it works as expected. I should have sprung for...,1356220800,23/12/2012,715,0,0
1,A3H99DFEG68SR,B007WTAJTO,1m2,This think has worked out great.Had a diff. br...,1384992000,21/11/2013,382,0,0
2,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"Bought it with Retail Packaging, arrived legit...",1373673600,13/07/2013,513,0,0
3,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,It's mini storage. It doesn't do anything els...,1367193600,29/04/2013,588,0,0
4,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,I have it in my phone and it never skips a bea...,1382140800,19/10/2013,415,0,0


In [4]:
# Vou usar o Groq por falta de capacidade no Gemini
# resposta = client.models.generate_content(model="gemini-2.5-flash", contents="O que é IA?")
# def analisar_sentimento(review):
#     try:
#         prompt = f"Responda apenas com a classificação de sentimento (Positivo, Negativo, Neutro) do review: {review}"
#
#         response = client.models.generate_content(
#             model="gemini-2.5-flash",
#             contents=prompt)
#
#         return response.text.strip()
#
#     except Exception as e:
#         print(f"Erro ao analisar sentimento: {e}")

In [15]:
from groq import Groq

client = Groq()

def analisar_sentimento(review):
    try:
        #prompt = f"Responda apenas com a classificação de sentimento (Positivo, Negativo, Neutro) do review: {review}"

        # FEW SHOT PROMPTING
        prompt = f"""Você irá analisar a resenha que eu te mandarei abaixo e retorna com uma análise de sentimento.
                    Você deve responder APENAS com uma das seguintes palavras: Positivo, Negativo ou Neutro, indicando o sentimento relativo àquela resenha específica.

                    Exemplos:
                    'This think has worked out great.Had a diff. bran 64gb card and if went south after 3 months.This one has held up pretty well since I had my S3, now on my Note3.*** update 3/21/14I\'ve had this for a few months and have had ZERO issue\'s since it was transferred from my S3 to my Note3 and into a note2. This card is reliable and solid!Cheers!' -> Positivo
                    'it works as expected. I should have sprung for the higher capacity.  I think its made a bit cheesier than the earlier versions; the paint looks not as clean as before' -> Negativo
                    'Bought it with Retail Packaging, arrived legit, in a orange envelope, english version not asian like the picture shows. arrived quickly, bought a 32 and 16 both retail packaging for my htc one sv and Lg Optimus, both cards in working order, probably best price you\'ll get for a nice sd card' -> Neutro

                    Resenha a ser analisada: {review}
                    """

        response = client.chat.completions.create(
            model="openai/gpt-oss-120b",
            messages=[
              {
                "role": "user",
                "content": prompt
              }
            ])
        print(response.choices[0].message.content)

        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Erro ao analisar sentimento: {e}")


In [16]:
df["review_sentimento"] = df["reviewText"].apply(analisar_sentimento)
df.head()

Negativo
Positivo
Positivo
Positivo
Positivo
Negativo
Negativo
Positivo
Negativo
Negativo
Negativo
Negativo
Negativo
Positivo
Negativo
Negativo
Positivo
Positivo
Negativo
Negativo
Neutro
Negativo
Negativo
Negativo
Negativo
Negativo
Negativo
Negativo
Neutro
Positivo
Negativo
Negativo
Positivo
Negativo
Negativo
Negativo
Negativo
Neutro
Negativo
Negativo


,reviewerID,asin,reviewerName,reviewText,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,review_sentimento
0,A2FII3I2MBMUIA,B007WTAJTO,1K3,it works as expected. I should have sprung for...,1356220800,23/12/2012,715,0,0,Negativo
1,A3H99DFEG68SR,B007WTAJTO,1m2,This think has worked out great.Had a diff. br...,1384992000,21/11/2013,382,0,0,Positivo
2,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"Bought it with Retail Packaging, arrived legit...",1373673600,13/07/2013,513,0,0,Positivo
3,A2IDCSC6NVONIZ,B007WTAJTO,2Cents!,It's mini storage. It doesn't do anything els...,1367193600,29/04/2013,588,0,0,Positivo
4,A26YHXZD5UFPVQ,B007WTAJTO,2K1Toaster,I have it in my phone and it never skips a bea...,1382140800,19/10/2013,415,0,0,Positivo
